<a href="https://colab.research.google.com/github/chanyoung1998/LearningML-DL/blob/main/5%EB%8B%A8%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#교차 검증과 그리드 서치(5-2단원)

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size = 0.2,random_state = 42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input,train_target,test_size = 0.2, random_state = 42)

In [ ]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


## 교차검증

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.01082039, 0.00693583, 0.00763869, 0.00725937, 0.00689125]), 'score_time': array([0.00070095, 0.00061154, 0.00065541, 0.00066233, 0.0005734 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


##그리드 서치


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state= 42),params, n_jobs = -1)
gs.fit(train_input,train_target)
dt = gs.best_estimator_
print(dt.score(train_input,train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


### 조금 더 복잡한 매개변수 조합


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state= 42),params,n_jobs= -1)
gs.fit(train_input,train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
from scipy.stats import uniform,randint

rgen = randint(0,10)
rgen.rvs(10)

array([3, 2, 1, 3, 2, 9, 3, 3, 8, 9])

#트리의 앙상블

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size = 0.2,random_state = 42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1,random_state = 42)
scores = cross_validate(rf,train_input,train_target,return_train_score = True,n_jobs = -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))
print(scores)

0.9973541965122431 0.8905151032797809
{'fit_time': array([0.57958841, 0.58012223, 0.67830229, 0.6773169 , 0.34120202]), 'score_time': array([0.10214305, 0.10329652, 0.10238481, 0.10353971, 0.10213757]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}


In [7]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)
print(rf.score(train_input,train_target))
print(rf.score(test_input,test_target))

[0.23167441 0.50039841 0.26792718]
0.996921300750433
0.8892307692307693


In [9]:
rf = RandomForestClassifier(oob_score= True, n_jobs= -1,random_state= 42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


#그레디언트 부스팅


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
scores = cross_validate(gb,train_input,train_target,return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators = 500,learning_rate = 0.2,random_state = 42)
scores = cross_validate(gb,train_input,train_target,return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


##히스토그램 기반 그레디언트 부스팅


In [19]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier()
scores = cross_validate(hgb,train_input,train_target,return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))
hgb.fit(train_input,train_target)
print(hgb.score(test_input,test_target))

0.9321723946453317 0.8801241948619236
0.8723076923076923
